### FIREHOSE
- Reliably load real-time streams into data lakes, warehouses, and analytics services

In [235]:
import boto3
from pandas import read_csv
from time import sleep

In [226]:
key_file = '/home/roberto/Documents/aws.csv'
df = read_csv(key_file)
key_file = '/home/roberto/Documents/awsFirehoseARN.csv'
dfARN =read_csv(key_file)

userName = 'roberto_casa'

AWS_KEY_ID = df[df.user==userName]['Access key ID'][0]
AWS_SECRET = df[df.user==userName]['Secret access key'][0]
AWS_ARN_FIREHOSE = dfARN['arnFirehose'][0]
AWS_ARN_S3 = dfARN.iloc[:,1][0]

firehose = boto3.client('firehose', region_name='us-east-1', 
                         aws_access_key_id=AWS_KEY_ID, 
                         aws_secret_access_key=AWS_SECRET)

s3 = boto3.client('s3', region_name='us-east-1', 
                         aws_access_key_id=AWS_KEY_ID, 
                         aws_secret_access_key=AWS_SECRET)


In [222]:
BucketName = 'pizza-orders'
s3.create_bucket(Bucket= bucketName)

# List the buckets in S3
for bucket_info in s3.list_buckets()['Buckets']:
    
    # Get the bucket_name
    bucket_name = bucket_info['Name']
    
    # Generate bucket ARN.
    arn = "arn:aws:s3:::{}".format(bucket_name)
    if 'pizza' in arn:
        AWS_BUCKET_PIZZA =arn
    # Print the ARN
    print(arn)

arn:aws:s3:::aws-emr-resources-408411726795-us-east-1
arn:aws:s3:::aws-logs-408411726795-us-east-1
arn:aws:s3:::databricks-workspace-stack-lambdazipsbucket-i3ixqoaq5qmb
arn:aws:s3:::db-d041ca200718732155ef996e3f967fa0-s3-root-bucket
arn:aws:s3:::pizza-orders
arn:aws:s3:::roberto-server


In [ ]:
streamName = 'orders-food'
res = firehose.create_delivery_stream(
    DeliveryStreamName = streamName,
    DeliveryStreamType = 'DirectPut',
    S3DestinationConfiguration = {
        "RoleARN" : AWS_ARN_FIREHOSE,
        "BucketARN" : AWS_BUCKET_PIZZA
    }
)

In [232]:
#show created delivery strams
response = firehose.list_delivery_streams()
print(response['DeliveryStreamNames'])

['orders-food']


In [230]:
print(res['DeliveryStreamARN'])

arn:aws:firehose:us-east-1:408411726795:deliverystream/orders-food


In [239]:
# OBD2_sensors.py: Write to Firehose stream. EDIT HERE.
for idx, row in records.iterrows(): 

    # Create a payload string that ends with a newline
    payload = ' '.join(str(value) for value in row) 
    payload = payload + "\n"
    print("Sending payload: {}".format(payload))

    # Send the payload string to Firehose stream
    res = firehose.put_record(
        DeliveryStreamName = 'gps-delivery-stream',
        Record = {'Data': payload})

    # Print the written RecordId
    print("Wrote to RecordId: {}".format(res['RecordId']))


NameError: name 'records' is not defined

In [229]:
#Delete them all!
for stream_name in response['DeliveryStreamNames']:
    firehose.delete_delivery_stream(DeliveryStreamName = stream_name)